In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.preprocessing import StandardScaler

tqdm.pandas(desc="my bar!")
pd.set_option("max_columns", 100)
pd.set_option("max_rows", 100)

def disp_full(x, drows=False, dcols=True):
    if drows:
        pd.set_option('display.max_rows', x.shape[0])
    if dcols:
        pd.set_option('display.max_columns', x.shape[1])
    display(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')


def wlogloss(targets, preds):
    target_cols = [
        "any",
        "epidural",
        "subdural",
        "subarachnoid",
        "intraventricular",
        "intraparenchymal",
    ]
    res = 0
    for col in target_cols:
        res += log_loss(targets[col], preds[col + "_pred"])
        if col == "any":
            res += log_loss(targets[col], preds[col + "_pred"])
    res /= 7
    return res


def make_feats(df):
    df["ImagePositionPatient_2"] = df["ImagePositionPatient"].progress_apply(
        lambda x: x[2]
    )
    df = df.merge(
        df.groupby(["StudyInstanceUID"])["ImagePositionPatient_2"]
        .agg(position_min="min", position_max="max")
        .reset_index(),
        on="StudyInstanceUID",
    )
    df["position"] = (df["ImagePositionPatient_2"] - df["position_min"]) / (
        df["position_max"] - df["position_min"]
    )
    res = df.sort_values(by=["StudyInstanceUID", "position"])
    return res


def pred_agg1_train(df):
    new_feats = []
    pred_cols = [column for column in df.columns if "pred" in column]
    print(pred_cols)
    for c in pred_cols:
        tmp = (
            df.groupby(["StudyInstanceUID"])[c]
            .agg(["min", "max", "mean", "std"])
            .reset_index()
        )
        tmp.columns = [
            "StudyInstanceUID",
            c + "_min",
            c + "_max",
            c + "_mean",
            c + "_std",
        ]
        if c != "any_pred_tsukiyama_inceotionv4":
            del tmp["StudyInstanceUID"]
        new_feats.append(tmp)
    new_feats = pd.concat(new_feats, axis=1)
    df = pd.merge(df, new_feats, on="StudyInstanceUID", how="left")
    for c in pred_cols:
        df[c + "_diff"] = df[c] - df[c + "_mean"]
        df[c + "_div"] = df[c] / df[c + "_mean"]
        df[c + "_scaled"] = (df[c] - df[c + "_mean"]) / df[c + "_std"]
    return df


def pred_agg1_test(df):
    new_feats = []
    pred_cols = [column for column in df.columns if "pred" in column]
    print(pred_cols)
    for c in pred_cols:
        tmp = (
            df.groupby(["StudyInstanceUID"])[c]
            .agg(["min", "max", "mean", "std"])
            .reset_index()
        )
        tmp.columns = [
            "StudyInstanceUID",
            c + "_min",
            c + "_max",
            c + "_mean",
            c + "_std",
        ]
        if "any_pred" not in c:
            del tmp["StudyInstanceUID"]
        new_feats.append(tmp)
    new_feats = pd.concat(new_feats, axis=1)
    df = pd.merge(df, new_feats, on="StudyInstanceUID", how="left")
    for c in pred_cols:
        df[c + "_diff"] = df[c] - df[c + "_mean"]
        df[c + "_div"] = df[c] / df[c + "_mean"]
        df[c + "_scaled"] = (df[c] - df[c + "_mean"]) / df[c + "_std"]
    return df


def pred_agg2(df):
    pred_cols = [column for column in df.columns if "pred" in column]
    print(pred_cols)
    a1 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(3, min_periods=1, center=True)
        .mean()
        .values
    )
    a2 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(5, min_periods=1, center=True)
        .mean()
        .values
    )
    a3 = (
        df.groupby("StudyInstanceUID")[pred_cols]
        .rolling(1, min_periods=1, center=True)
        .mean()
        .values
    )
    new_feats1 = pd.DataFrame(a1, columns=[c + "_3roll" for c in pred_cols])
    new_feats2 = pd.DataFrame(a2, columns=[c + "_5roll" for c in pred_cols])
    new_feats3 = pd.DataFrame(a1 - a3, columns=[c + "_3rolldiff" for c in pred_cols])
    new_feats4 = pd.DataFrame(a2 - a3, columns=[c + "_5rolldiff" for c in pred_cols])
    new_feats5 = pd.DataFrame(a1 / a3, columns=[c + "_3rolldiv" for c in pred_cols])
    new_feats6 = pd.DataFrame(a2 / a3, columns=[c + "_5rolldiv" for c in pred_cols])
    new_feats1.index = df.index
    new_feats2.index = df.index
    new_feats3.index = df.index
    new_feats4.index = df.index
    new_feats5.index = df.index
    new_feats6.index = df.index
    df = pd.concat(
        [df, new_feats1, new_feats2, new_feats3, new_feats4, new_feats5, new_feats6],
        axis=1,
    )
    return df

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
train = pd.read_pickle('data_for_stacking/cache/train_raw.pkl')
train = make_feats(train)
test = pd.read_pickle("data_for_stacking/cache/test_raw.pkl")
test = make_feats(test)

my bar!: 100%|██████████| 78545/78545 [00:00<00:00, 440781.17it/s]


In [2]:
target_cols = [
    "any",
    "epidural",
    "subdural",
    "subarachnoid",
    "intraventricular",
    "intraparenchymal",
]
models = ['tsukiyama_inceotionv4',
          'sasaki_se_resnext101_mixup_410_with_imagenet_norm',
          'sasaki_se_resnext_410',
          "appian",
          "sasaki_senet154_customlabels",
          "sugawara_efficientnetb3",
          "tsukiyama_xception_224",
          "tsukiyama_inception_resnet_v2_336",
          "shimakoshi_seresnext101_adj_prediction",
          "shimakoshi_seresnext50_label_smoothing"
         ]

In [4]:
# train

model_dfs = []
for model in tqdm(models):
    df_all = []
    n_tta = 5
    for n_fold in range(5):
        df = pd.read_pickle(f"data_for_stacking/{model}/fold{n_fold}_ep2_valid_tta5.pkl")
        if "shimakoshi" not in model:
            tmp = np.zeros([len(df[0]["ids"]), 6])
            for i in range(n_tta):
                tmp += df[i]["outputs"] / n_tta
        else:
            tmp = df["outputs"]
        tmp = pd.DataFrame(tmp)
        tmp.columns = [tar_col + "_pred" for tar_col in target_cols]
        if "shimakoshi" not in model:
            tmp["ID"] = df[0]["ids"]
            target = df[0]["targets"]
        else:
            tmp["ID"] = df["ids"]
            target = df["targets"]
        tmp["folds"] = n_fold
        if model == "sasaki_senet154_customlabels":
            target[target<0.5] = 0
        tmp2 = pd.DataFrame(target)
        tmp2.columns = [tar_col + "_true" for tar_col in target_cols]
        df_all.append(pd.concat([tmp, tmp2], axis=1))
    df_all = pd.concat(df_all)
    model_dfs.append(df_all)

columns = list(model_dfs[0].columns)
print(columns)
columns.remove("ID")
for i, model_df in enumerate(model_dfs):
    rename_dict = {}
    for column in columns:
        rename_dict[column] = column+"_{}".format(i)
    if i == 0:
        df = model_df.rename(columns=rename_dict)
    else:
        df = pd.merge(df, model_df.rename(columns=rename_dict), on="ID", how="outer")


100%|██████████| 10/10 [00:33<00:00,  3.32s/it]


['any_pred', 'epidural_pred', 'subdural_pred', 'subarachnoid_pred', 'intraventricular_pred', 'intraparenchymal_pred', 'ID', 'folds', 'any_true', 'epidural_true', 'subdural_true', 'subarachnoid_true', 'intraventricular_true', 'intraparenchymal_true']


In [5]:
pred_columns = [c for c in df.columns if "pred" in c]
df[pred_columns] = df[pred_columns].clip(1e-15, 1-1e-15)

In [6]:
for i, row in df[df.isnull().any(axis=1)].iterrows():
    for column in columns:
        print(i)
        tmp = [c for c in df.columns if column in c]
        nan_columns = []
        for j, t in enumerate(row[tmp].isnull()):
            if t:
                nan_columns.append(tmp[j])
        tmp = row[tmp].mean()
        df.loc[i, nan_columns] = tmp

127860
127860
127860
127860
127860
127860
127860
127860
127860
127860
127860
127860
127860
647822
647822
647822
647822
647822
647822
647822
647822
647822
647822
647822
647822
647822


In [7]:
n_model = len(model_dfs)
model_dfs = []
for i in range(n_model):
    tmp_columns = [c for c in df.columns if "_{}".format(i) in c]
    df_all = df[tmp_columns+["ID"]]
    rename_dict = {}
    for tmp_column in tmp_columns:
        rename_dict[tmp_column] = tmp_column.replace("_{}".format(i), "").replace("_true", "")
    df_all = df_all.rename(columns=rename_dict)
    model_dfs.append(df_all)
    
for i, model_df in enumerate(model_dfs):
    rename_dict = {}
    for target_col in target_cols:
        rename_dict[target_col+"_pred"] = target_col + "_pred_" + models[i]
    if i == 0:
        df_all = model_df.rename(columns=rename_dict)
    else:
        c = model_df.columns.drop(target_cols+["folds", ])
        df_all = pd.merge(df_all, model_df[c].rename(columns=rename_dict), on="ID", how="left")
print(df_all.columns)

Index(['any_pred_tsukiyama_inceotionv4', 'epidural_pred_tsukiyama_inceotionv4',
       'subdural_pred_tsukiyama_inceotionv4',
       'subarachnoid_pred_tsukiyama_inceotionv4',
       'intraventricular_pred_tsukiyama_inceotionv4',
       'intraparenchymal_pred_tsukiyama_inceotionv4', 'folds', 'any',
       'epidural', 'subdural', 'subarachnoid', 'intraventricular',
       'intraparenchymal', 'ID',
       'any_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'epidural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subdural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'subarachnoid_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraventricular_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'intraparenchymal_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm',
       'any_pred_sasaki_se_resnext_410', 'epidural_pred_sasaki_se_resnext_410',
       'subdural_pred_sasaki_se_resnext_410',
       'subarac

In [8]:
df_all = df_all.merge(train, on='ID')
if "position" in df_all.columns:
    df_all = df_all.sort_values(['StudyInstanceUID', 'position'])
else:
    df_all = df_all.sort_values(['StudyInstanceUID'])
    
df_all = pred_agg1_train(df_all)
df_all = pred_agg2(df_all)

['any_pred_tsukiyama_inceotionv4', 'epidural_pred_tsukiyama_inceotionv4', 'subdural_pred_tsukiyama_inceotionv4', 'subarachnoid_pred_tsukiyama_inceotionv4', 'intraventricular_pred_tsukiyama_inceotionv4', 'intraparenchymal_pred_tsukiyama_inceotionv4', 'any_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm', 'epidural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm', 'subdural_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm', 'subarachnoid_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm', 'intraventricular_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm', 'intraparenchymal_pred_sasaki_se_resnext101_mixup_410_with_imagenet_norm', 'any_pred_sasaki_se_resnext_410', 'epidural_pred_sasaki_se_resnext_410', 'subdural_pred_sasaki_se_resnext_410', 'subarachnoid_pred_sasaki_se_resnext_410', 'intraventricular_pred_sasaki_se_resnext_410', 'intraparenchymal_pred_sasaki_se_resnext_410', 'any_pred_appian', 'epidural_pred_appian', 'subdural_pred_appian', 'subarachnoid_pre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
#with open("df_all.pkl", "wb") as f:
#    pickle.dump(df_all, f)
#with open("df_all.pkl", "rb") as f:
#    df_all = pickle.load(f)

In [9]:
# test

model_df_list_test = []
for model in tqdm(models):
    df_all_test = []
    for n_fold in range(5):
        df = pd.read_pickle(f"data_for_stacking/{model}/fold{n_fold}_ep2_test_tta5.pkl")
        if "shimakoshi" not in model:
            tmp = np.zeros([len(df[0]["ids"]), 6])
            for i in range(n_tta):
                tmp += df[i]["outputs"] / n_tta
        else:
            tmp = df["outputs"]
        tmp = pd.DataFrame(tmp)
        tmp.columns = [tar_col + "_pred" for tar_col in target_cols]
        if "shimakoshi" not in model:
            tmp["ID"] = df[n_fold]["ids"]
        else:
            tmp["ID"] = df["ids"]
        tmp["folds"] = n_fold
        pred_columns = [c for c in tmp.columns if "pred" in c]
        tmp[pred_columns] = tmp[pred_columns].clip(1e-15, 1-1e-15)
        df_all_test.append(tmp)
    model_df_list_test.append(df_all_test)


100%|██████████| 10/10 [00:23<00:00,  2.36s/it]


In [10]:
columns_test = list(model_df_list_test[0][0].columns)
columns_test.remove("ID")
print(columns_test)
averaged_df_list_test = []
for j in range(len(model_df_list_test[0])):
    for i in range(len(model_df_list_test)):
        rename_dict = {}
        for column in columns_test:
            rename_dict[column] = column+"_{}".format(i)
        if i == 0:
            tmp_df = model_df_list_test[i][j].rename(columns=rename_dict)
        else:
            #tmp_df = pd.merge(tmp_df, model_df_list_test[i][j].drop(columns=["folds"]), on="ID", how="outer")
            tmp_df = pd.merge(tmp_df, model_df_list_test[i][j].rename(columns=rename_dict), on="ID", how="outer")
    averaged_df_list_test.append(tmp_df)

['any_pred', 'epidural_pred', 'subdural_pred', 'subarachnoid_pred', 'intraventricular_pred', 'intraparenchymal_pred', 'folds']


In [11]:
for j, averaged_df in enumerate(averaged_df_list_test):
    for i, row in averaged_df[averaged_df.isnull().any(axis=1)].iterrows():
        print(i)
        for column in columns_test:
            print(j, i, column)
            tmp = [c for c in averaged_df.columns if column in c]
            nan_columns = []
            for k, t in enumerate(row[tmp].isnull()):
                if t:
                    nan_columns.append(tmp[k])
            tmp = row[tmp].mean()
            averaged_df_list_test[j].loc[i, nan_columns] = tmp

In [12]:
n_model = len(model_df_list_test)
model_df_list_test = [[] for i in range(n_model)]
for j, averaged_df in enumerate(averaged_df_list_test):
    for i in range(n_model):
        tmp_columns = [c for c in averaged_df.columns if "_{}".format(i) in c]
        df_all_test = averaged_df[tmp_columns+["ID"]]
        rename_dict = {}
        for tmp_column in tmp_columns:
            rename_dict[tmp_column] = tmp_column.replace("_{}".format(i), "").replace("_true", "")
        df_all_test = df_all_test.rename(columns=rename_dict)
        model_df_list_test[i].append(df_all_test)

In [13]:
for j, model_dfs_test in enumerate(model_df_list_test):
    for i, model_df in enumerate(model_dfs_test):
        rename_dict = {}
        for target_col in target_cols:
            rename_dict[target_col+"_pred"] = target_col + "_pred_" + models[j]
        model_df = model_df.rename(columns=rename_dict)
        model_df = model_df.merge(test, on="ID")
        if "position" in model_df.columns:
            model_df = model_df.sort_values(['StudyInstanceUID', 'position'])
        else:
            model_df = model_df.sort_values(['StudyInstanceUID'])
        print(model_df.columns)
        model_df = pred_agg1_test(model_df)
        model_df = pred_agg2(model_df)
        model_df_list_test[j][i] = model_df


Index(['any_pred_tsukiyama_inceotionv4', 'epidural_pred_tsukiyama_inceotionv4',
       'subdural_pred_tsukiyama_inceotionv4',
       'subarachnoid_pred_tsukiyama_inceotionv4',
       'intraventricular_pred_tsukiyama_inceotionv4',
       'intraparenchymal_pred_tsukiyama_inceotionv4', 'folds', 'ID', 'labels',
       'n_label', 'BitsAllocated', 'BitsStored', 'Columns', 'HighBit',
       'ImageOrientationPatient', 'ImagePositionPatient', 'Modality',
       'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
       'PixelSpacing', 'RescaleIntercept', 'RescaleSlope', 'Rows',
       'SOPInstanceUID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID',
       'StudyInstanceUID', 'WindowCenter', 'WindowWidth', 'raw_max', 'raw_min',
       'raw_mean', 'raw_diff', 'doctor_max', 'doctor_min', 'doctor_mean',
       'doctor_diff', 'custom_max', 'custom_min', 'custom_mean', 'custom_diff',
       'ImagePositionPatient_2', 'position_min', 'position_max', 'position'],
      dtype='object')
[

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide


Index(['any_pred_shimakoshi_seresnext101_adj_prediction',
       'epidural_pred_shimakoshi_seresnext101_adj_prediction',
       'subdural_pred_shimakoshi_seresnext101_adj_prediction',
       'subarachnoid_pred_shimakoshi_seresnext101_adj_prediction',
       'intraventricular_pred_shimakoshi_seresnext101_adj_prediction',
       'intraparenchymal_pred_shimakoshi_seresnext101_adj_prediction', 'folds',
       'ID', 'labels', 'n_label', 'BitsAllocated', 'BitsStored', 'Columns',
       'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient',
       'Modality', 'PatientID', 'PhotometricInterpretation',
       'PixelRepresentation', 'PixelSpacing', 'RescaleIntercept',
       'RescaleSlope', 'Rows', 'SOPInstanceUID', 'SamplesPerPixel',
       'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 'WindowCenter',
       'WindowWidth', 'raw_max', 'raw_min', 'raw_mean', 'raw_diff',
       'doctor_max', 'doctor_min', 'doctor_mean', 'doctor_diff', 'custom_max',
       'custom_min', 'custom_mean', 

In [133]:
#with open("model_df_list_test.pkl", "wb") as f:
#    pickle.dump(model_df_list_test, f)
#with open("model_df_list_test.pkl", "rb") as f:
#    model_df_list_test = pickle.load(f)

In [13]:
#object colはone hot encoding

from sklearn.preprocessing import OneHotEncoder

 """
 
 encode_candidate_cols = df_all.columns.drop(
    ['StudyInstanceUID', 'ID', 'folds', 'labels', 'n_label', 'SOPInstanceUID', 'PatientID', 'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID']
    + target_cols)
 
for c in encode_candidate_cols:
    if df_all[c].dtype == 'O' and c != "ImagePositionPatient":
        print(c)
        tmp = pd.concat([df_all[c], model_df_list_test[0][0][c]]).reset_index(drop=True)
        #tmp = df_all[c]
        tmp = pd.factorize(tmp)[0]
        n_label = tmp.max()+1
        ohe = OneHotEncoder(sparse=False)
        tmp = ohe.fit_transform(tmp.reshape(-1, 1))
        c_names = [c+"_{}".format(j) for j in range(n_label)]
        tmp = pd.DataFrame(tmp, columns=c_names)
        df_all = pd.concat([df_all, tmp[:len(df_all)]], axis=1).drop(columns=[c])
        for i in range(len(model_df_list_test)):
            for j in range(len(model_df_list_test[i])):
                model_df_list_test[i][j] = pd.concat([model_df_list_test[i][j], tmp[len(df_all):].reset_index(drop=True)], axis=1).drop(columns=[c])

    elif c == "ImagePositionPatient":
        # こいつだけ数が多すぎるのでlabel encoding
        print(c)
        tmp = pd.concat([df_all[c], model_df_list_test[0][0][c]]).reset_index(drop=True)
        tmp = pd.factorize(tmp)[0]
        df_all[c] = tmp[:len(df_all)]
        for i in range(len(model_df_list_test)):
            for j in range(len(model_df_list_test[i])):
                tmp_df = model_df_list_test[i][j]
                tmp_df[c] = tmp[len(df_all):]
                model_df_list_test[i][j] = tmp_df
"""

ImageOrientationPatient


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ImagePositionPatient
Modality


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


PhotometricInterpretation


...

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


WindowCenter


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


WindowWidth


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
# 6: target
# 30: 共通のパラメータ
# 507: 共通のパラメータ起因の特徴量
# 336: pred起因の特徴量

print(len(df_all.columns)), print(len(model_df_list_test[0][0].columns))

30+507+336, 6+30+507+336*7

3370
340


(873, 2895)

In [14]:
X_cols = df_all.columns.drop([c for c in df_all.columns if "pred" not in c]+target_cols)

df_all = df_all[list(X_cols)+target_cols+["ID", "folds", "StudyInstanceUID", "position"]]
for i in range(len(model_df_list_test)):
    for j in range(len(model_df_list_test[0])):
        cols = model_df_list_test[i][j].columns
        cols = [col for col in cols if col in list(X_cols)+["ID", "folds", "StudyInstanceUID", "position"]]
        model_df_list_test[i][j] = model_df_list_test[i][j][cols]

In [15]:
n_models = len(model_df_list_test)
n_folds = len(model_df_list_test[0])
test_cols = set([])
for i in range(n_models):
    test_cols = test_cols | set(model_df_list_test[i][0].columns)

test_list = []
for j in range(n_folds):
    tmp_test_cols = test_cols
    for i in range(n_models):
        cols = model_df_list_test[i][j].columns
        use_cols = [col for col in cols if col in tmp_test_cols]
        if i == 0:
            df = model_df_list_test[i][j][use_cols]
        else:
            df = pd.merge(df, model_df_list_test[i][j][use_cols+["ID", ]], on="ID", how="outer")
        tmp_test_cols = tmp_test_cols - set(use_cols)
        print(len(tmp_test_cols))
    test_list.append(df)


3024
2688
2352
2016
1680
1344
1008
672
336
0
3024
2688
2352
2016
1680
1344
1008
672
336
0
3024
2688
2352
2016
1680
1344
1008
672
336
0
3024
2688
2352
2016
1680
1344
1008
672
336
0
3024
2688
2352
2016
1680
1344
1008
672
336
0


In [3]:
#with open("df_all_preprcessed_10models.pkl", "wb") as f:
#    pickle.dump(df_all, f, protocol=4)
#with open("test_list_preprocessed_10models.pkl", "wb") as f:
#    pickle.dump(test_list, f, protocol=4)
with open("df_all_preprcessed_10models.pkl", "rb") as f:
    df_all = pickle.load(f)
with open("test_list_preprocessed_10models.pkl", "rb") as f:
    test_list = pickle.load(f)


# CNN

In [20]:
# 6: target
# 10: 共通の捨てるパラメータ
# 20: 共通の使うパラメータ
# 507: 共通のパラメータ起因の特徴量
# 336: pred起因の特徴量

print(len(df_all.columns)), print(len(model_df_list_test[0][0].columns)), print(len(test_list[0].iloc[0]))

# target+["ID", "folds", "StudyInstanceUID", "position"]+pred由来
6+4+336*8


3370
340
3364


2698

In [4]:
columns_all = list(df_all.columns.drop(['StudyInstanceUID', 'ID', 'folds', 'position']))
print(len(columns_all))
columns_models = []
for model in models:
    columns_model = [c for c in df_all.columns if model in c]
    columns_model = [c for c in columns_model if c[-3:] != "div"]
    columns_models.append(columns_model)
    columns_all = [c for c in columns_all if c not in columns_model]
columns_intersection = set(columns_all) - set(target_cols)
print(len(columns_intersection), [len(columns_model) for columns_model in columns_models], len(target_cols))

3366
1020 [234, 234, 234, 234, 234, 234, 234, 234, 234, 234] 6


In [5]:
df_all = df_all.sort_values(["folds", 'StudyInstanceUID', 'position']).reset_index(drop=True)
for i in range(len(test_list)):
    test_list[i] = test_list[i].sort_values(['StudyInstanceUID', 'position']).reset_index(drop=True)

In [6]:
X_train_list = []
ids_train_list = []
#X_train_intersection_list = []
X_valid_list = []
ids_valid_list = []
#X_valid_intersection_list = []
for i in range(len(test_list)):
    df_train = df_all[df_all.folds!=i]
    tmp_train = np.empty((len(models), len(df_train), len(columns_models[0])))
    df_valid = df_all[df_all.folds==i]
    tmp_valid = np.empty((len(models), len(df_valid), len(columns_models[0])))
    for j in range(len(models)):
        tmp_train[j] = df_train[columns_models[j]]
        tmp_valid[j] = df_valid[columns_models[j]]
    X_train_list.append(tmp_train)
    ids_train_list.append(df_train.ID.to_list())
    #X_train_intersection_list.append(df_train[list(columns_intersection)].values)
    X_valid_list.append(tmp_valid)
    ids_valid_list.append(df_valid.ID.to_list())
    #X_valid_intersection_list.append(df_valid[list(columns_intersection)].values)

In [7]:
# n_folds, n_models, n_samples, n_features
print(len(X_train_list), len(X_train_list[0]), len(X_train_list[0][0]), len(X_train_list[0][0][0]), len(ids_train_list), len(ids_train_list[0]))
print(len(X_valid_list), len(X_valid_list[0]), len(X_valid_list[0][0]), len(X_valid_list[0][0][0]), len(ids_valid_list), len(ids_valid_list[0]))

# n_folds, n_samples, n_features
#print(len(X_train_intersection_list), len(X_train_intersection_list[0]), len(X_train_intersection_list[0][0]))
#print(len(X_valid_intersection_list), len(X_valid_intersection_list[0]), len(X_valid_intersection_list[0][0]))

5 10 531554 234 5 531554
5 10 133860 234 5 133860


In [8]:
train_batch_size_list = []
for i in range(5):
    train_batch_size_list.append(df_all[df_all.folds==i].groupby(["StudyInstanceUID"]).apply(lambda x: x.shape[0]).values)

test_batch_size = test_list[0].groupby(["StudyInstanceUID"]).apply(lambda x: x.shape[0]).values

In [9]:
all_train_batch_counter = []

for i in range(5):
    train_batch_num = 0
    train_batch_counter = []
    for j, train_batch_size in enumerate(train_batch_size_list):
        if i != j:
            for train_batch_n in train_batch_size:
                train_batch_counter.extend([train_batch_num, ]*train_batch_n)
                train_batch_num += 1
    all_train_batch_counter.append(train_batch_counter)

all_valid_batch_counter = []
for i in range(5):
    valid_batch_num = 0
    valid_batch_counter = []
    for j in train_batch_size_list[i]:
        valid_batch_counter.extend([valid_batch_num, ]*j)
        valid_batch_num += 1
    all_valid_batch_counter.append(valid_batch_counter)


In [10]:
test_batch_num = 0
test_batch_counter = []
for j in test_batch_size:
    test_batch_counter.extend([test_batch_num, ]*j)
    test_batch_num += 1


In [11]:
y_train_list = []
y_valid_list = []
for i in range(5):
    df = df_all[df_all.folds!=i]
    y_train_list.append(df[target_cols].values)
    df = df_all[df_all.folds==i]
    y_valid_list.append(df[target_cols].values)


In [12]:
# n_folds, n_samples, n_target
print(len(y_train_list), len(y_train_list[0]), len(y_train_list[0][0]))
print(len(y_valid_list), len(y_valid_list[0]), len(y_valid_list[0][0]))

5 531554 6
5 133860 6


In [13]:
X_test = np.empty((len(test_list), len(models), len(test_list[0]), len(columns_models[0])))
X_test_list = []
ids_test_list = []
#X_test_intersection_list = []
for i in range(len(test_list)):
    tmp_test = np.empty((len(models), len(test_list[0]), len(columns_models[0])))
    for j in range(len(models)):
        tmp_test[j] = test_list[i][columns_models[j]]
    X_test_list.append(tmp_test)
    ids_test_list.append(test_list[i].ID.to_list())
    #X_test_intersection_list.append(test_list[i][list(columns_intersection)].values)

In [14]:
# n_folds, n_models, n_samples, n_features
print(len(X_test_list), len(X_test_list[0]), len(X_test_list[0][0]), len(X_test_list[0][0][0]), len(ids_test_list), len(ids_test_list[0]))
# n_folds, n_samples, n_features
#print(len(X_test_intersection_list), len(X_test_intersection_list[0]), len(X_test_intersection_list[0][0]))

5 10 78363 234 5 78363


In [15]:
#dataset = (X_train_list, X_valid_list, y_train_list, y_valid_list, X_test_list)
#with open("dataset.pkl", "wb") as f:
#    pickle.dump(dataset, f, protocol=4)

In [76]:
"""
for i in range(5):
    for j in range(X_train_list[i].shape[0]):
        scaler = StandardScaler()
        X_train_list[i][j] = scaler.fit_transform(X_train_list[i][j])
        X_valid_list[i][j] = scaler.transform(X_valid_list[i][j])
        X_test_list[i][j] = scaler.transform(X_test_list[i][j])
"""

In [15]:
# model 8と9はnanが多かった
for i in range(5):
    X_train_list[i] = X_train_list[i][:8]
    X_valid_list[i] = X_valid_list[i][:8]
    X_test_list[i] = X_test_list[i][:8]

In [16]:
for i, X_train in enumerate(X_train_list):
    X_train_list[i] = X_train.transpose(1, 0, 2)
for i, X_valid in enumerate(X_valid_list):
    X_valid_list[i] = X_valid.transpose(1, 0, 2)
for i, X_test in enumerate(X_test_list):
    X_test_list[i] = X_test.transpose(1, 0, 2)

In [24]:
"""
class StackingDataset(Dataset):
    def __init__(self, X, X_intersection, y):
        self.X = X
        self.X_intersection = X_intersection
        self.y = y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.X_intersection[idx]), torch.FloatTensor(self.y[idx])
    
class StackingDatasetTest(Dataset):
    def __init__(self, X, X_intersection):
        self.X = X
        self.X_intersection = X_intersection
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor(self.X_intersection[idx])
"""

class StackingContinuousDataset(Dataset):
    def __init__(self, X, batch_num_list, ids_list, y):
        self.X = X
        self.y = y
        self.ids_list = ids_list
        self.batch_num_list = batch_num_list
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        if idx == 0:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]]), self.ids_list[idx], torch.FloatTensor(self.y[idx])
        elif idx ==  len(self.batch_num_list)-1:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]]), self.ids_list[idx], torch.FloatTensor(self.y[idx])
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1] and self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx, idx, idx]]), self.ids_list[idx], torch.FloatTensor(self.y[idx])
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1]:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]]), self.ids_list[idx], torch.FloatTensor(self.y[idx])
        elif self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]]), self.ids_list[idx], torch.FloatTensor(self.y[idx])
        else:
            return torch.FloatTensor(self.X[[idx-1, idx, idx+1]]), self.ids_list[idx], torch.FloatTensor(self.y[idx])

class StackingContinuousDatasetTest(Dataset):
    def __init__(self, X, batch_num_list, ids_list):
        self.X = X
        self.ids_list = ids_list
        self.batch_num_list = batch_num_list
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        if idx == 0:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]]), self.ids_list[idx]
        elif idx == len(self.batch_num_list)-1:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]]), self.ids_list[idx]
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1] and self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx, idx, idx]]), self.ids_list[idx]
        elif self.batch_num_list[idx] != self.batch_num_list[idx-1]:
            return torch.FloatTensor(self.X[[idx, idx, idx+1]]), self.ids_list[idx]
        elif self.batch_num_list[idx] != self.batch_num_list[idx+1]:
            return torch.FloatTensor(self.X[[idx-1, idx, idx]]), self.ids_list[idx]
        else:
            return torch.FloatTensor(self.X[[idx-1, idx, idx+1]]), self.ids_list[idx]

In [43]:
"""
def get_loader(fold, mode):
    if mode == "train":
        train_loader = DataLoader(
            StackingDataset(X_train_list[fold], X_train_intersection_list[fold], y_train_list[fold]),
            shuffle=True,
            batch_size=128,
            num_workers=8,
        #    pin_memory=True,
        )
        return train_loader
    elif mode == "valid":
        valid_loader = DataLoader(
            StackingDataset(X_valid_list[fold], X_valid_intersection_list[fold], y_valid_list[fold]),
            batch_size=128,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return valid_loader
    elif mode == "test":
        test_loader = DataLoader(
            StackingDatasetTest(X_test_list[fold], X_test_intersection_list[fold]),
            batch_size=128,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return test_loader
    else:
        assert False
"""

def get_loader(fold, mode):
    if mode == "train":
        train_loader = DataLoader(
            StackingContinuousDataset(X_train_list[fold], all_train_batch_counter[fold], ids_train_list[fold], y_train_list[fold]),
            shuffle=True,
            batch_size=512,
            num_workers=8,
        #    pin_memory=True,
        )
        return train_loader
    elif mode == "valid":
        valid_loader = DataLoader(
            StackingContinuousDataset(X_valid_list[fold], all_valid_batch_counter[fold], ids_valid_list[fold], y_valid_list[fold]),
            batch_size=512,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return valid_loader
    elif mode == "test":
        test_loader = DataLoader(
            StackingContinuousDatasetTest(X_test_list[fold], test_batch_counter, ids_test_list[fold]),
            batch_size=512,
            shuffle=False,
            num_workers=8,
        #    pin_memory=True,
        )
        return test_loader
    else:
        assert False

In [44]:
#np.random.seed(seed=100)
#swap_columns_idx = [np.random.permutation(np.arange(len(models))) for i in range(8)]


class StackingModel(nn.Module):
    def __init__(self):
        super(StackingModel, self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(4, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 1), padding=(0, 0), bias=False),
            nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(2, 1), padding=(0, 0), bias=False),
            #nn.ReLU(inplace=True),
        )
        """
        self.dense_intersection_layer = nn.Sequential(
            nn.Linear(527, 256),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.Linear(256, 256),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.Linear(256, 256),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
        )
        """
        self.dense_concat_layer = nn.Sequential(
            nn.Linear(7488, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 6),

#            nn.ReLU(inplace=True),
#            nn.Linear(1024, 128),
#            nn.ReLU(inplace=True),
#            nn.Linear(128, 6),
        )
        
    def forward(self, x):
        #x, x_intersection = xs
        #for i in range(8):
        #    if i == 0:
        #        shuffled_xs = x[:, :, swap_columns_idx[i], :]
        #    else:
        #        shuffled_xs = torch.cat((shuffled_xs, x[:, :, swap_columns_idx[i], :]), 1)
        #x = self.cnn_layer(shuffled_xs)
        out = self.cnn_layer(x)
        out = out.view(x.shape[0], -1)
        #x_intersection = self.dense_intersection_layer(x_intersection)
        #x_intersection = x_intersection.view(x.shape[0], -1)
        #out = torch.cat((x, x_intersection), 1)
        out = self.dense_concat_layer(out)
        #x = torch.clamp(x, min=1e-15, max=1-1e-15)
        return out

In [45]:
print(StackingModel())

StackingModel(
  (cnn_layer): Sequential(
    (0): Conv2d(3, 8, kernel_size=(4, 1), stride=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): Conv2d(8, 16, kernel_size=(3, 1), stride=(1, 1), bias=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(16, 32, kernel_size=(3, 1), stride=(1, 1), bias=False)
    (5): ReLU(inplace=True)
  )
  (dense_concat_layer): Sequential(
    (0): Linear(in_features=7488, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=6, bias=True)
  )
)


In [97]:
pred_test = []
pred_test_ids = []
pred_valid = []
pred_valid_ids = []
for i in range(5):
    print("fold {}/5\n".format(i+1))
    tr = get_loader(i, "train")
    va = get_loader(i, "valid")
    te = get_loader(i, "test")
    
    model = StackingModel().cuda()
    optimizer = torch.optim.Adam(model.parameters(), 2e-4)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1, 3, 6, 9, 12], gamma=0.5)
    criterion = nn.BCEWithLogitsLoss(weight=torch.FloatTensor([2, 1, 1, 1, 1, 1]).cuda())
    
    n_epoch = 15
    pred_valid_fold = []
    pred_valid_ids_fold = []
    for epoch in range(n_epoch):
        model.train()
        losses = []
        for x, ids, y in tr:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses.append(loss.item())

        losses_valid = []
        with torch.no_grad():
            model.eval()
            for x, ids, y in va:                
                pred = model(x.cuda())
                loss = criterion(pred, y.cuda())
                losses_valid.append(loss.item())
                pred_valid_ids_fold.extend(ids)
                if epoch == n_epoch-1:
                    pred_valid_fold.append(torch.sigmoid(pred).detach().cpu().numpy())
            print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))
        scheduler.step()

    with torch.no_grad():
        model.eval()
        pred_test_fold = []
        pred_test_ids_fold = []
        for x, ids in te:            
            pred = model(x.cuda())
            pred_test_fold.append(torch.sigmoid(pred).detach().cpu().numpy())
            pred_test_ids_fold.extend(ids)
    pred_test.append(pred_test_fold)
    pred_test_ids.append(pred_test_ids_fold)
    pred_valid.append(pred_valid_fold)
    pred_valid_ids.append(pred_valid_fold)
    
    torch.save({"model": model.state_dict(), "optimizer": optimizer.state_dict()}, "fold{}.pt".format(i))

fold 1/5



Process Process-1556:
Process Process-1553:
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fdce85e3830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


KeyboardInterrupt: 

Process Process-1559:
Process Process-1557:
Process Process-1560:
Process Process-1555:
Process Process-1554:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/

In [99]:
"""
with open("pred_test.pkl", "wb") as f:
    pickle.dump(pred_test, f)
with open("pred_valid.pkl", "wb") as f:
    pickle.dump(pred_valid, f)
with open("pred_test_ids.pkl", "wb") as f:
    pickle.dump(pred_test_ids, f)
with open("pred_pred_valid_ids.pkl", "wb") as f:
    pickle.dump(pred_valid_ids, f)
"""
with open("pred_test.pkl", "rb") as f:
    pred_test = pickle.load(f)
with open("pred_valid.pkl", "rb") as f:
    pred_valid = pickle.load(f)
with open("pred_test_ids.pkl", "rb") as f:
    pred_test_ids = pickle.load(f)
with open("pred_pred_valid_ids.pkl", "rb") as f:
    pred_valid_ids = pickle.load(f)


In [73]:
array_pred_valid = []
for i in range(5):
    tmp = np.zeros((1, 6))
    for j in range(len(pred_valid[i])):
        tmp = np.concatenate((tmp, pred_valid[i][j]), axis=0)
    array_pred_valid.append(tmp[1:])
    

In [81]:
array_pred_test = []
for i in range(5):
    tmp = np.zeros((1, 6))
    for j in range(len(pred_test[i])):
        tmp = np.concatenate((tmp, pred_test[i][j]), axis=0)
    array_pred_test.append(tmp[1:])
    

In [114]:
for i in range(5):
    with open("cnn_fold{}_valid_pred.pkl".format(i), "wb") as f:
        pickle.dump({"ids": ids_valid_list[i], "outputs": array_pred_valid[i]}, f)


In [115]:
#for i in range(5):
#    with open("cnn_fold{}_test_pred.pkl".format(i), "wb") as f:
#        pickle.dump({"ids": ids_test_list[i], "outputs": array_pred_test[i]}, f)

In [101]:
print(len(pred_valid_ids), len(pred_valid_ids[0]), len(pred_valid_ids[0][0]), len(pred_valid_ids[0][0][0]))
print(len(pred_test_ids), len(pred_test_ids[0]), len(pred_test_ids[0][0]), len(pred_test_ids[0][0][0]))

5 262 512 6
5 78363 12 1


In [139]:
sub_id_list = pd.read_csv("../data/stage_1_sample_submission.csv")
ids = []
for i in range(len(sub_id_list)):
    if i%6 == 0:
        ids.append(sub_id_list[i].replace("_epidural", ""))

In [166]:
for i in range(5):
    sub = pd.DataFrame(ids, columns=["ID"])
    test_sub = pd.DataFrame(array_pred_test[i], columns=target_cols)
    test_sub["ID"] = pred_test_ids[i]
    sub = pd.merge(sub, test_sub, on="ID", how="left")
    sub = sub.fillna(min(sub[target_cols].min()))
    with open("cnn_fold{}_test_pred.pkl".format(i), "wb") as f:
        pickle.dump({"ids": sub.ID.tolist(), "outputs": sub[target_cols].values}, f)

In [86]:
class StackingModel2(nn.Module):
    def __init__(self):
        super(StackingModel2, self).__init__()
        self.dense_layer = nn.Sequential(
            nn.Linear(7056, 1000),
            nn.Tanh(),
            nn.Linear(1000, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 6),
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], 7056)
        out = self.dense_layer(x)
        return out

In [87]:
model = StackingModel2().cuda()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
criterion = nn.BCEWithLogitsLoss(weight=torch.FloatTensor([2,1,1,1,1,1]).cuda())

In [88]:
print(model)

StackingModel2(
  (dense_layer): Sequential(
    (0): Linear(in_features=7056, out_features=1000, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1000, out_features=200, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=200, out_features=6, bias=True)
  )
)


In [89]:
pred_test = []
for i in range(5):
    print("fold {}/5\n".format(i+1))
    tr = get_loader(i, "train")
    va = get_loader(i, "valid")
    te = get_loader(i, "test")
    for epoch in range(10):
        print(epoch)
        model.train()
        losses = []
        for x, y in tr:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            loss.backward()
            optimizer.step()
            
            optimizer.zero_grad()
            losses.append(loss.item())

        losses_valid = []
        with torch.no_grad():
            model.eval()
            for x, y in va:
                pred = model(x.cuda())
                loss = criterion(pred, y.cuda())
                losses_valid.append(loss.item())
            print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))

    with torch.no_grad():
        model.eval()
        for x in te:
            pred = model(x.cuda())
            pred_test.append(pred)

fold 1/5

0
Epoch[0] Train: nan Valid: nan
1


KeyboardInterrupt: 

In [24]:
model.train()
for epoch in range(10):
    losses = []
    for x, y in train_loader:
        pred = model(x.cuda())
        loss = criterion(pred, y.cuda())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
    losses_valid = []
    with torch.no_grad():
        model.eval()
        for x, y in valid_loader:
            pred = model(x.cuda())
            loss = criterion(pred, y.cuda())
            losses_valid.append(loss.item())
    print(f'Epoch[{epoch}]', 'Train:', np.mean(losses), 'Valid:', np.mean(losses_valid))
    model.train()

Epoch[0] Train: 0.24050997862371834 Valid: 0.16861760471272105
Epoch[1] Train: 0.1602081820329327 Valid: 0.15870382193163152
Epoch[2] Train: 0.15492199151933594 Valid: 0.15616315897582597
Epoch[3] Train: 0.1529936831932124 Valid: 0.15505525547982626
Epoch[4] Train: 0.15195317921713639 Valid: 0.1544597020216343
Epoch[5] Train: 0.15126592678459821 Valid: 0.1540926653493679
Epoch[6] Train: 0.15077115383440193 Valid: 0.15387332139408771
Epoch[7] Train: 0.15038474744384797 Valid: 0.15371600970739627
Epoch[8] Train: 0.1500640829005014 Valid: 0.1535945014445154
Epoch[9] Train: 0.14979854242246923 Valid: 0.15351682496150487


In [ ]:
y

In [ ]:
criterion(pred, y.cuda()).item()

In [32]:
disp_full(model_dfs[0])

,any_pred,epidural_pred,subdural_pred,subarachnoid_pred,intraventricular_pred,intraparenchymal_pred,folds,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal,ID,StudyInstanceUID,PositionOrd,any_min,any_max,any_mean,any_std,epidural_min,epidural_max,epidural_mean,epidural_std,subdural_min,subdural_max,subdural_mean,subdural_std,subarachnoid_min,subarachnoid_max,subarachnoid_mean,subarachnoid_std,intraventricular_min,intraventricular_max,intraventricular_mean,intraventricular_std,intraparenchymal_min,intraparenchymal_max,intraparenchymal_mean,intraparenchymal_std,any_diff,any_div,any_scaled,epidural_diff,epidural_div,epidural_scaled,subdural_diff,subdural_div,subdural_scaled,subarachnoid_diff,subarachnoid_div,subarachnoid_scaled,intraventricular_diff,intraventricular_div,intraventricular_scaled,intraparenchymal_diff,intraparenchymal_div,intraparenchymal_scaled,any_3roll,epidural_3roll,subdural_3roll,subarachnoid_3roll,intraventricular_3roll,intraparenchymal_3roll,any_5roll,epidural_5roll,subdural_5roll,subarachnoid_5roll,intraventricular_5roll,intraparenchymal_5roll,any_3rolldiff,epidural_3rolldiff,subdural_3rolldiff,subarachnoid_3rolldiff,intraventricular_3rolldiff,intraparenchymal_3rolldiff,any_5rolldiff,epidural_5rolldiff,subdural_5rolldiff,subarachnoid_5rolldiff,intraventricular_5rolldiff,intraparenchymal_5rolldiff,any_3rolldiv,epidural_3rolldiv,subdural_3rolldiv,subarachnoid_3rolldiv,intraventricular_3rolldiv,intraparenchymal_3rolldiv,any_5rolldiv,epidural_5rolldiv,subdural_5rolldiv,subarachnoid_5rolldiv,intraventricular_5rolldiv,intraparenchymal_5rolldiv
0,0.000051,2.000906e-06,0.000009,0.000019,4.938827e-06,0.000002,0,0.0,0.0,0.0,0.0,0.0,0.0,ID_3a422b8d7,ID_00047d6503,0.025000,0.000048,0.305901,0.039593,0.058922,1.646989e-06,0.090424,0.008224,0.016279,0.000009,0.242983,0.027816,0.045110,0.000015,0.029519,0.005339,0.006859,1.239594e-06,0.011768,0.001143,0.002657,0.000002,0.062310,0.004936,0.010124,-0.039541,0.001300,-0.671077,-0.008222,0.000243,-0.505105,-0.027808,0.000308,-0.616448,-0.005320,0.003493,-0.775595,-0.001138,0.004320,-0.428411,-0.004934,0.000506,-0.487321,0.000057,0.000003,0.000012,0.000018,0.000005,0.000004,0.000258,0.000023,0.000047,0.000122,0.000035,0.000029,0.000005,0.000001,0.000004,-6.081101e-07,-1.911224e-07,0.000001,0.000206,0.000021,0.000038,0.000103,0.000030,0.000026,1.103592,1.640546,1.446989,0.967389,0.961302,1.515170,5.008629,11.420777,5.434845,6.517519,6.999029,11.482114
1,0.000062,4.564251e-06,0.000016,0.000017,4.556583e-06,0.000005,0,0.0,0.0,0.0,0.0,0.0,0.0,ID_490b10d5a,ID_00047d6503,0.050000,0.000048,0.305901,0.039593,0.058922,1.646989e-06,0.090424,0.008224,0.016279,0.000009,0.242983,0.027816,0.045110,0.000015,0.029519,0.005339,0.006859,1.239594e-06,0.011768,0.001143,0.002657,0.000002,0.062310,0.004936,0.010124,-0.039531,0.001569,-0.670896,-0.008220,0.000555,-0.504947,-0.027800,0.000583,-0.616278,-0.005321,0.003265,-0.775772,-0.001139,0.003985,-0.428555,-0.004931,0.001028,-0.487066,0.000258,0.000023,0.000047,0.000122,0.000035,0.000029,0.001391,0.000083,0.000374,0.000589,0.000232,0.000268,0.000196,0.000018,0.000030,1.041049e-04,3.001041e-05,0.000024,0.001329,0.000079,0.000358,0.000571,0.000227,0.000263,4.149021,5.006713,2.869538,6.972257,7.586166,5.655266,22.391411,18.277673,23.067760,33.769063,50.842242,52.751340
2,0.000660,6.199053e-05,0.000115,0.000329,9.420557e-05,0.000079,0,0.0,0.0,0.0,0.0,0.0,0.0,ID_be2a0ca1c,ID_00047d6503,0.075000,0.000048,0.305901,0.039593,0.058922,1.646989e-06,0.090424,0.008224,0.016279,0.000009,0.242983,0.027816,0.045110,0.000015,0.029519,0.005339,0.006859,1.239594e-06,0.011768,0.001143,0.002657,0.000002,0.062310,0.004936,0.010124,-0.038933,0.016661,-0.660755,-0.008162,0.007537,-0.501420,-0.027702,0.004129,-0.614092,-0.005010,0.061539,-0.730417,-0.001049,0.082396,-0.394817,-0.004858,0.015910,-0.479810,0.001837,0.000111,0.000496,0.000779,0.000307,0.000356,0.003206,0.000142,0.000703,0.001872,0.000482,0.000533,0.001178,0.000049,0.000381,4.501109e-04,2.13

In [34]:
csv = pd.read_csv("../data/stage_1_train.csv")
n_train = 4045572//6

name = []
value = []
for i in tqdm(range(n_train), total=n_train):
    name.append(csv.iloc[i*6].ID.replace("_epidural", ""))
    value.append(csv.iloc[i*6:(i+1)*6].Label.to_list())

100%|██████████| 674262/674262 [04:31<00:00, 2481.66it/s]


In [86]:
csv_df = pd.DataFrame(value, columns=["epidural", "intraparenchymal", "intraventricular", "subarachnoid", "subdural", "any"])
csv_df["ID"] = list(map(lambda x: x.replace("_epidural", ""), name))
csv_df = csv_df[["ID", "any", "epidural", "subdural", "subarachnoid", "intraventricular", "intraparenchymal"]]
display(csv_df)

,ID,any,epidural,subdural,subarachnoid,intraventricular,intraparenchymal
0,ID_63eb1e259,0,0,0,0,0,0
1,ID_2669954a7,0,0,0,0,0,0
2,ID_52c9913b1,0,0,0,0,0,0
3,ID_4e6ff6126,0,0,0,0,0,0
4,ID_7858edd88,0,0,0,0,0,0
...,...,...,...,...,...,...,...
674257,ID_f737f4cc1,0,0,0,0,0,0
674258,ID_4c92d70b5,0,0,0,0,0,0
674259,ID_a9797cb3a,0,0,0,0,0,0
674260,ID_9375f67bd,0,0,0,0,0,0
